In [1]:
import random
#random.seed(42)  

In [2]:
import math
import cv2
import numpy as np
import os 
import glob 
from sklearn.cluster import KMeans
from sklearn import svm
from functools import reduce
import operator
from sklearn import metrics
from IPython.core.display import Image, display
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.ensemble import IsolationForest


In [3]:
# defining feature extractor that we want to use
extractor = cv2.xfeatures2d.SIFT_create()
def features(image, extractor):
    keypoints, descriptors = extractor.detectAndCompute(image, None)
    return keypoints, descriptors

In [4]:
def build_histogram(descriptor_list, cluster_alg):
    
    histogram = np.zeros(len(cluster_alg.cluster_centers_))
    cluster_result =  cluster_alg.predict(descriptor_list)
    for i in cluster_result:
        histogram[i] += 1.0
    return  histogram / len(cluster_result)

In [5]:
def onedossier(i):
 file_dir = 'C:\\Users\\LENOVO\\damage//*'
 files = glob.glob(file_dir)
 descriptor_list=np.empty(shape=[1, 128])
 x=[]
 descriptor=[] 
 img_dir = os.path.join(files[i],'*')   
 images = glob.glob(img_dir)
 if (len(images)==0):
    print('empty folder')    
 else:   
    for img in images:
        img_temp = cv2.imread(img)
        img_temp=cv2.cvtColor(img_temp, cv2.COLOR_BGR2GRAY)
        img_temp = cv2.resize(img_temp,(512,512))
        x.append(img_temp)
        keypoint,descriptors = features(img_temp, extractor)
        if(descriptors is not None):
         descriptors[~np.isnan(descriptors).any(axis=1)]  
         descriptor_list=np.concatenate((descriptor_list, descriptors),axis=0) 
                
 return  descriptor_list,x

In [ ]:
file_dir = 'C:\\Users\\LENOVO\\damage//*'
files = glob.glob(file_dir)

In [ ]:
def validation(i,kmeans):
 val_image2=[]
 #random.seed(4)  
 for q in range(len(files)):
  if (q!=i)  :
   img_dir = os.path.join(files[q],'*')
   images = glob.glob(img_dir)
   sampling = random.choices(images, k=1)
   print('this is sampling:',sampling)
   data = cv2.imread(sampling[0])
   data =cv2.cvtColor(data, cv2.COLOR_BGR2GRAY)
   data =cv2.resize(data,(512,512))
   keypoint, descriptors = features(data, extractor)  
   descriptors[~np.isnan(descriptors).any(axis=1)]
   histogram = build_histogram(descriptors, kmeans)
   val_image2.append(histogram)  
 return val_image2 

In [ ]:
index=[x for x in range(15)]
random.shuffle(index)
num1=index[:14]
print('this is num1:',num1)
num2=index[14:]
print('this is num2:',num2)

this is num1: [10, 9, 14, 3, 0, 7, 6, 12, 11, 13, 1, 4, 5, 2]
this is num2: [8]


In [ ]:
def optimize_training_parameters(i):
        scoreinit=0
        opt_estim =8
        opt_samples=8
        opt_contamin=0.1
        opt_feat=1
        opt_boots=False
        opt_n_jo=3
        opt_nbclusters=111
        random.seed(4)  
        for k in [111,241,150,70]:
         if (scoreinit<1):
          kmeans = KMeans(n_clusters =k,random_state=42)
          print('opt',i)
          [l1,l2]=onedossier(i)
          l1=np.nan_to_num(l1)
          l1[~np.isnan(l1).any(axis=1)]
          kmeans.fit(l1) 
          preprocessed_image=[]  
          for f in (l2): 
                 
                 keypoint,descriptors = features(f, extractor)  
                 if(descriptors is not None):
                     descriptors[~np.isnan(descriptors).any(axis=1)]
                     histogram = build_histogram(descriptors, kmeans)
                     preprocessed_image.append(histogram)
                     
        
         
        

         
          preprocessed_image1=[preprocessed_image[b] for b in num1]
          val_image1=[preprocessed_image[b] for b in num2]
          target1=np.ones(len(val_image1)) 
          out=validation(i,kmeans)
          val_image2=out[:4]
          target2=-1*np.ones(len(val_image2))
          val_image=val_image1+val_image2
          target=np.concatenate((target1,target2))
          random.seed(4)  
          for number in [100,50,200,600,5,11]:
           if (scoreinit<1):
            for samples in [100,150,200]:
                if (scoreinit<1):
                  for contam in [0.01,0.02,0.1,0.2,0.3,0.4]:
                    if (scoreinit<1):
                        for feat in [50,100,5,10,11,1]:
                         if (scoreinit<1):
                          for  boots in  [True, False] :
                           if (scoreinit<1):
                            for n_jo in [5, 10, 20, 30]:
                             classifier = IsolationForest(behaviour="new",n_estimators=number, max_samples=samples, contamination=contam, max_features=feat, bootstrap=boots, n_jobs=n_jo)
                             classifier.fit(preprocessed_image1)
                             preds = classifier.predict(val_image)
                             print(type(preds))
                             #accuracy= metrics.accuracy_score(target, preds)
                             accuracy= metrics.precision_score(target, preds)
                             y_pred = classifier.predict(preprocessed_image1)
                        
                             print(y_pred)
                             print(preds)
                             print(target)   
                             print('this is acc',accuracy)  
                             print('this is sc',scoreinit) 
                             if (scoreinit < accuracy):
                            
                                        scoreinit=accuracy
                                        trainig_img=preprocessed_image1
                                        opt_estim = number 
                                        opt_samples=samples
                                        opt_contamin=contam
                                        opt_feat=feat
                                        opt_boots=boots
                                        opt_n_jo=n_jo
                                        
                                        if (scoreinit==0.8):
                                           break
        print('out of loop')                        
                                   
        return  opt_estim ,opt_samples,opt_contamin,opt_feat,opt_boots,opt_n_jo, opt_nbclusters,trainig_img

In [ ]:
list4=[]
listopt=[]
for z in range(2):
     print(z) 
     #random.seed(42)     
     M=optimize_training_parameters(z)
     listopt.append(M)   
     print('finish')
     kmeans = KMeans(n_clusters =listopt[z][6],random_state=42)
     [l1,l2]=onedossier(z) 
     l1=np.nan_to_num(l1)   
     l1[~np.isnan(l1).any(axis=1)]  
     kmeans.fit(l1)
     list4.append(kmeans)   

0
opt 0
this is sampling: ['C:\\Users\\LENOVO\\damage\\damage10\\12.jpg']
this is sampling: ['C:\\Users\\LENOVO\\damage\\damage2\\10.jpg']
this is sampling: ['C:\\Users\\LENOVO\\damage\\damage3\\14.jpg']
this is sampling: ['C:\\Users\\LENOVO\\damage\\damage4\\11.jpg']
this is sampling: ['C:\\Users\\LENOVO\\damage\\damage5\\1.JPG']
this is sampling: ['C:\\Users\\LENOVO\\damage\\damage6\\15.jpg']
this is sampling: ['C:\\Users\\LENOVO\\damage\\damage7\\8.jpg']
this is sampling: ['C:\\Users\\LENOVO\\damage\\damage8\\7.jpg']
this is sampling: ['C:\\Users\\LENOVO\\damage\\damage9\\6.jpg']


C:\Users\LENOVO\anaconda2018\lib\site-packages\sklearn\ensemble\iforest.py:250: UserWarning: max_samples (100) is greater than the total number of samples (14). max_samples will be set to n_samples for estimation.
  % (self.max_samples, n_samples))


<class 'numpy.ndarray'>
[ 1  1  1  1  1  1 -1  1  1  1  1  1  1  1]
[1 1 1 1 1]
[ 1. -1. -1. -1. -1.]
this is acc 0.2
this is sc 0


C:\Users\LENOVO\anaconda2018\lib\site-packages\sklearn\ensemble\iforest.py:250: UserWarning: max_samples (100) is greater than the total number of samples (14). max_samples will be set to n_samples for estimation.
  % (self.max_samples, n_samples))


<class 'numpy.ndarray'>
[ 1  1  1  1  1  1 -1  1  1  1  1  1  1  1]
[1 1 1 1 1]
[ 1. -1. -1. -1. -1.]
this is acc 0.2
this is sc 0.2


...

C:\Users\LENOVO\anaconda2018\lib\site-packages\sklearn\ensemble\iforest.py:250: UserWarning: max_samples (100) is greater than the total number of samples (14). max_samples will be set to n_samples for estimation.
  % (self.max_samples, n_samples))


<class 'numpy.ndarray'>
[ 1  1 -1  1  1  1  1  1  1 -1  1  1  1  1]
[ 1 -1 -1  1  1]
[ 1. -1. -1. -1. -1.]
this is acc 0.3333333333333333
this is sc 0.5


C:\Users\LENOVO\anaconda2018\lib\site-packages\sklearn\ensemble\iforest.py:250: UserWarning: max_samples (100) is greater than the total number of samples (14). max_samples will be set to n_samples for estimation.
  % (self.max_samples, n_samples))


<class 'numpy.ndarray'>
[ 1  1 -1  1  1  1  1  1  1 -1  1  1  1  1]
[ 1 -1 -1  1  1]
[ 1. -1. -1. -1. -1.]
this is acc 0.3333333333333333
this is sc 0.5


C:\Users\LENOVO\anaconda2018\lib\site-packages\sklearn\ensemble\iforest.py:250: UserWarning: max_samples (100) is greater than the total number of samples (14). max_samples will be set to n_samples for estimation.
  % (self.max_samples, n_samples))


In [ ]:
print(listopt)
print(num1)
print(num2)
print(list4)

In [ ]:
file_dir ='C:\\Users\\LENOVO\\nouveau images/*'
file = glob.glob(file_dir)
print(file)
print(len(file))

In [ ]:
myorder = [0, 1, 3, 4, 5,6,7,9,10,12,2,8,11,13]
print(len(file))
file = [file[h] for h in myorder]
print(file)

In [ ]:
def test(g):
 test_image=[]
 for f1 in file:
  data = cv2.imread(f1)
  data =cv2.cvtColor(data, cv2.COLOR_BGR2GRAY)
  data =cv2.resize(data,(512,512))
  keypoint, descriptors = features(data, extractor)  
  descriptors[~np.isnan(descriptors).any(axis=1)]  
  histogram = build_histogram(descriptors, list4[g])
  test_image.append(histogram) 
 return(test_image)  

In [ ]:
list2=[]
list3=[]
#for i in range(len(files)):
for i in range(2):    
 
  if((listopt[i][0]!=None)&(listopt[i][1]!=None)&(listopt[i][2]!=None)&(listopt[i][3]!=None)&(listopt[i][4]!=None)&(listopt[i][5]!=None)&(listopt[i][6]!=None)&(listopt[i][7]!=None)): 
     clf =IsolationForest(behaviour="new",n_estimators=listopt[i][0], max_samples=M[1], contamination=listopt[i][2], max_features=listopt[i][3], bootstrap=listopt[i][4], n_jobs=listopt[i][5])
     clf.fit(listopt[i][7])
     y_pred_train = clf.predict(listopt[i][7])
     print(y_pred_train )
     print('avant')
     y_pred_test = clf.predict(test(i))
     print('apres')
  list3.append(y_pred_train)
  list2.append(y_pred_test)
print('khouloud')

In [ ]:
print('this is the training pred',list3)
print('this is the test pred',list2)